In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data1=pd.read_excel('Processed Data\Intrinsic Data.xlsx', sheet_name=0, usecols="B:E,H:I")
data2=pd.read_excel('Processed Data\Extrinsic Data.xlsx', sheet_name=0, usecols="B:P")

In [3]:
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder()

encoded_data1=pd.DataFrame(encoder.fit_transform(data1[['Breed']]).toarray())

Y1_=data1['Milk Production/Day']
data1=data1.drop(['Breed','Milk Production/Day'],axis=1)

data1=data1.join(encoded_data1)
data1=data1.join(Y1_)

encoded_data2=pd.DataFrame(encoder.fit_transform(data2[['Month','District','Zone','Green \nFodder','Dry\n Fodder']]).toarray())
Y2_=data2['Farm Milk Production per \nDay']
data2=data2.drop(['Month','District','Zone','Green \nFodder','Dry\n Fodder','Farm Milk Production per \nDay'],axis=1)

data2=data2.join(encoded_data2)
data2=data2.join(Y2_)

data2=data2.to_numpy()
X2=data2[:,0:-1]
Y2=data2[:,-1]

In [4]:
data1=data1.to_numpy()
X1=data1[:,0:-1]
Y1=data1[:,-1]

In [5]:
Tdata1=pd.read_excel('TDataIn.xlsx', sheet_name=0, usecols="B:E,H:I")
Tdata2=pd.read_excel('TDataEx.xlsx', sheet_name=0, usecols="B:P")

from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder()

Tencoded_data1=pd.DataFrame(encoder.fit_transform(Tdata1[['Breed']]).toarray())

TY1_=Tdata1['Milk Production/Day']
Tdata1=Tdata1.drop(['Breed','Milk Production/Day'],axis=1)

Tdata1=Tdata1.join(Tencoded_data1)
Tdata1=Tdata1.join(TY1_)

print(Tdata1.tail(10))

Tencoded_data2=pd.DataFrame(encoder.fit_transform(Tdata2[['Month','District','Zone','Green \nFodder','Dry\n Fodder']]).toarray())
TY2_=Tdata2['Farm Milk Production per \nDay']
Tdata2=Tdata2.drop(['Month','District','Zone','Green \nFodder','Dry\n Fodder','Farm Milk Production per \nDay'],axis=1)

Tdata2=Tdata2.join(Tencoded_data2)
Tdata2=Tdata2.join(TY2_)

# print(Tdata1)
# print(Tdata2)

Tdata2=Tdata2.to_numpy()
TX2=Tdata2[:,0:-1]
TY2=Tdata2[:,-1]

Tdata1=Tdata1.to_numpy()
TX1=Tdata1[:,0:-1]
TY1=Tdata1[:,-1]

# print(TX1)
# print(TX2)

farmSize=[]
for i in TX2:
    farmSize.append(i[8])
      
    

    Farm Size   Age  Parity/Lactation no.  Age of 1st Calving    0    1    2  \
55          4   7.0                     4                 3.0  0.0  0.0  1.0   
56          4   7.0                     4                 2.5  0.0  0.0  1.0   
57          4   7.5                     5                 3.0  0.0  0.0  0.0   
58          5  10.0                     4                 3.0  0.0  0.0  0.0   
59          5   9.5                     4                 3.5  0.0  0.0  0.0   
60          5   7.0                     3                 3.5  0.0  0.0  0.0   
61          5   5.5                     2                 3.5  0.0  0.0  0.0   
62          5   4.0                     1                 3.5  0.0  0.0  0.0   
63          1   6.0                     4                 2.5  0.0  1.0  0.0   
64          1   7.0                     3                 4.0  0.0  0.0  0.0   

      3    4    5  Milk Production/Day  
55  0.0  0.0  0.0                  8.0  
56  0.0  0.0  0.0                  7.

In [6]:
from sklearn.model_selection import train_test_split

X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1, Y1, test_size = 0.01, random_state = 0)
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y2, test_size = 0.01, random_state = 0)

X1_test=TX1
Y1_test=TY1

X2_test=X2[-18:]
Y2_test=TY2

print(Y2_test)

from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import PolynomialFeatures 
from sklearn.metrics import mean_squared_error,r2_score
# print(X2_test)


# For Intrinsic.
poly = PolynomialFeatures(degree=1, include_bias=True)
x_train_trans = poly.fit_transform(X1_train)
x_test_trans = poly.transform(X1_test)
lr = LinearRegression()
lr.fit(x_train_trans, Y1_train)
y_pred = lr.predict(x_test_trans)
# print(y_pred)

# For Extrinsic.
poly = PolynomialFeatures(degree=1, include_bias=True)
x_train_trans2 = poly.fit_transform(X2_train)
x_test_trans2 = poly.transform(X2_test)
lr2 = LinearRegression()
lr2.fit(x_train_trans2, Y2_train)
y_pred2 = lr2.predict(x_test_trans2)

print(lr.intercept_)
print(lr.coef_)

# print(r2_score(Y2_test, y_pred2))


[ 10.   11.   20.   47.    4.5  15.   23.  100.   18.   13.   20.   14.
  16.    8.   20.   17.    9.    3. ]
3.5562231567465625
[ 0.          0.04514957 -0.57395445  0.61455102  1.59916944 -0.66777694
  2.79924935  0.26971087 -0.82022929 -1.32007493 -0.26087906]


In [7]:
weight=-1
error=6000000


from sklearn.metrics import mean_squared_error,r2_score

for i in range(0,101):
    wIn=i
    wEx=100-i
    lower=0
    y_predictW=[]
    for idx in range(len(farmSize)):
        upper=lower+int(farmSize[idx])
        sum=0
        for farmIdx in range(lower,upper):
            sum+=y_pred[farmIdx]

        weightedAvg=(sum*wIn+y_pred2[idx]*wEx)/100
        
        for farmIdx in range(lower,upper):
            y_predictW.append(y_pred[farmIdx]*(weightedAvg/sum))
        lower=upper 
        
    if error > mean_squared_error(y_predictW,Y1_test):
        error = mean_squared_error(y_predictW,Y1_test)
        weight=i;
    
print(error)
print(weight)

4.219764428639967
91
